In [1]:
import pandas as pd
import ETL_functions as etf
import ast

# ETL

en este archivo haare todo el procesamiento de los archivos que se me dieron para generar archivos procesados de forma basica y archivos especificos para cada consulta necesaria

primero trabajare para desanidar y obtener unos archivos que me sean utiles para generar los archivos para cada consulta a partir de :

* user_reviews.json
* user_items.json

al archivo steam_games.json lo trabajare especificamente al momento de crear los dataframe finales para cada funcion

<hr>

# ETL  user_reviews.json

    cargo el archivo

In [48]:
reviews = []

with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", encoding='utf-8') as f:
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
df_ur = pd.DataFrame(reviews)

    transformo las siguiente columna a tipo str ya que es dificil manejar listas y tira errores en bastantes metodos de pandas

In [ ]:
df_ur["reviews"] = df_ur["reviews"].astype(str)

In [ ]:
etf.data_review(df_ur)

## valores nulos

    hay listas vacias que pasamos a str y por lo tanto no cuentan como nulos pero igualmente tendremos que borrar despues

In [ ]:
mask = df_ur["reviews"] == "[]"

In [ ]:
df_ur[mask]

    estas son todas las reviews con columna reviews nulos no las borrar ya que estaria borrando la fila y no quiero perder los registros de usuarios que podrian servir para normalizar otros archivos

de este archivo pienso sacar 2 ya que no es nesesario saber quien hizo la review puedo separarlo en 2 dataframe distintos :

uno con la parte de  informacion de el usuario que puede usarse para normalizar otras columnas

otro con las reviews desaanidas y el analisis de sentimiento

    para crear el primer archivo voy a dropear la columna reviews

In [51]:
del df_ur["reviews"]

In [52]:
etf.data_review(df_ur)


Total rows:  25799

Total full null rows:  0

Total duplicated rows: 314


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,user_id,[<class 'str'>],100.0,25799,0.0,0
1,user_url,[<class 'str'>],100.0,25799,0.0,0


    vemos que tenemos filas duplicadas  las  que tendremos que borrar

In [53]:
df_ur.drop_duplicates(inplace=True)

## dataframe resultante
|columna |tipo|
|--------|----|
|user_id | str|
|user_url| str|

## guardamos en un archivo parquet

In [55]:
df_ur.to_parquet(r"../Datasets/user_data.parquet")#dentro de la carpeta datasets

    para el segundo archivo tendremos que volver a importar el archivo a un dataaframe pero esta vez conservar solo la columna reviews

In [56]:
reviews = []

with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", encoding='utf-8') as f:
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
df_ur = pd.DataFrame(reviews)

    probando como se comporta la columna anidada frente al metodo de pandas pd.json_normalize

In [57]:
pd.json_normalize(df_ur["reviews"])#al hacer json_normalize nos  devuelve de esta forma un df de diccionarios

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
25794,"{'funny': '', 'posted': 'Posted May 31.', 'las...",None,None,None,None,None,None,None,None,None
25795,"{'funny': '', 'posted': 'Posted June 17.', 'la...",None,None,None,None,None,None,None,None,None
25796,"{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None,None
25797,"{'funny': '', 'posted': 'Posted July 21.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 8.', 'las...",None,None,None,None,None,None


In [58]:
pd.json_normalize(df_ur["reviews"][0])# al hacer json_normalize en cada fila nos devuelve el siguiente dataframe que es lo que me interesa

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


    basado en este comportamiento me di cuenta de que lo puedo maanejar a travez de iteraciones y json_normaalizer cada que necesite acceder a sus valores lo que me creara un dataframe dentro de un bucle con la columna que yo le pida metodo que voy a utlilizar en el user_items dado  su gran tamaño y en este caso al ser mucho mas ligero puedo permitirme crear un dataframe a partir de todos estos datos

### bucle para obtener todos los diccionarios en un dataframe llamaado df_sentimientos

    al hacer esto perdemos quien hizo la review (dato que no nos es necesario) y conservamos el id de el juego 

In [59]:
df_sentimientos = pd.DataFrame(columns=["funny","posted","last_edited","item_id","helpful","recommend","review"])

for i in df_ur["reviews"]:
    var = pd.json_normalize(i)
    # Supongamos que df1 y df2 son tus DataFrames con las mismas columnas
    df_sentimientos = pd.concat([df_sentimientos, var], axis=0)

df_sentimientos

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
2,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
3,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
0,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
1,,Posted July 20.,,730,No ratings yet,True,:D


solo conservaremos las  columnas:

* posted(para tener referencia a una fecha)
* item_id (para  relacionar con otros dataframes)
* recommend(para saber si se recomienda o no un juego)
* review (columna que usaremos para el aanalisis de sentimiento)


In [60]:
del df_sentimientos["funny"]
del df_sentimientos["helpful"]
del df_sentimientos["last_edited"]

    mostramos el archivo sin estas columnas que no utilizaremos

In [61]:
df_sentimientos

,posted,item_id,recommend,review
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
0,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
2,Posted July 10.,70,True,a must have classic from steam definitely wort...
3,Posted July 8.,362890,True,this game is a perfect remake of the original ...
0,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
1,Posted July 20.,730,True,:D


    asi nos quedaria el dataframe que usaremos para analisis  de sentimientos posted se podria borrar pero pienso dejarla por el momento

In [62]:
etf.data_review(df_sentimientos)


Total rows:  59305

Total full null rows:  0

Total duplicated rows: 881


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,posted,[<class 'str'>],100.0,59305,0.0,0
1,item_id,[<class 'str'>],100.0,59305,0.0,0
2,recommend,[<class 'bool'>],100.0,59305,0.0,0
3,review,[<class 'str'>],100.0,59305,0.0,0


    borro las filas duplicadas

In [67]:
df_sentimientos.drop_duplicates(inplace=True)

In [68]:
df_sentimientos.duplicated().sum()#ya no contamos con duplicados

0

## guardar el archivo como parquet

In [70]:
df_sentimientos.to_parquet(r"../Datasets/reviews_dict.parquet")

<hr>

#  ETL user_items.json

## cargamos el archivo

In [8]:
items = []

with open(r'../Datasets/users_items.json/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        items.append(ast.literal_eval(line))

df_ui = pd.DataFrame(items)

## usamos dos bucles para desanidar la lista de json anidados y guardar los valores en nuevas columnas

In [9]:
desanidado = []
for index, row in df_ui.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    user_items_count = row['items_count']
    for df_items_dict in row['items']:
        df_items_dict['user_id'] = user_id
        df_items_dict['user_url'] = user_url
        df_items_dict['items_count'] = user_items_count
        desanidado.append(df_items_dict)

df_ui = pd.DataFrame(desanidado)

    estes el dataframe resultante de la ejecucion de los bucles

In [10]:
df_ui

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,user_url,items_count
0,10,Counter-Strike,6,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
1,20,Team Fortress Classic,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
2,30,Day of Defeat,7,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
3,40,Deathmatch Classic,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
4,50,Half-Life: Opposing Force,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153205,373330,All Is Dust,0,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7


In [12]:
etf.data_review(df_ui)


Total rows:  5153209

Total full null rows:  0

Total duplicated rows: 59104


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,item_id,[<class 'str'>],100.0,5153209,0.0,0
1,item_name,[<class 'str'>],100.0,5153209,0.0,0
2,playtime_forever,[<class 'int'>],100.0,5153209,0.0,0
3,playtime_2weeks,[<class 'int'>],100.0,5153209,0.0,0
4,user_id,[<class 'str'>],100.0,5153209,0.0,0
5,user_url,[<class 'str'>],100.0,5153209,0.0,0
6,items_count,[<class 'int'>],100.0,5153209,0.0,0


    podemos notar que tenemos filas duplicadas

In [13]:
df_ui.drop_duplicates(inplace=True)#borramos las filas duplicadas

    ahora si revisamos ya no tenemos duplicados

In [14]:
df_ui.duplicated().sum()

0

In [16]:
df_ui.shape#estas son las filas sin de el dataframe sin duplicados y las columnas

(5094105, 7)

In [37]:
df_ui.sample(7)

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,user_url,items_count
183091,282440,Quake Live,0,0,76561198141880878,http://steamcommunity.com/profiles/76561198141...,55
2317699,730,Counter-Strike: Global Offensive,26264,0,weoweoman,http://steamcommunity.com/id/weoweoman,209
4369819,218230,PlanetSide 2,900,0,76561198046110507,http://steamcommunity.com/profiles/76561198046...,41
2660037,65930,The Bureau: XCOM Declassified,7,0,ImJustPro,http://steamcommunity.com/id/ImJustPro,247
3632741,265670,Imagine Me,83,0,TuDragon76a,http://steamcommunity.com/id/TuDragon76a,1269
256698,203160,Tomb Raider,852,0,76561197970812298,http://steamcommunity.com/profiles/76561197970...,149
4298878,3620,Zuma's Revenge,0,0,76561198042684717,http://steamcommunity.com/profiles/76561198042...,147


    podemos decir que playtime_2weeks no la vamos a usar teniendo el total de tiempo jugado(playtime_forever) 

In [38]:
del df_ui["playtime_2weeks"]#borramos la columna 

In [39]:
df_ui.sample(3)

,item_id,item_name,playtime_forever,user_id,user_url,items_count
613858,377160,Fallout 4,2325,76561198055528217,http://steamcommunity.com/profiles/76561198055...,69
3369360,385200,Absconding Zatwor,408,-SEVEN-,http://steamcommunity.com/id/-SEVEN-,394
2223941,730,Counter-Strike: Global Offensive,210,thesexyjesus,http://steamcommunity.com/id/thesexyjesus,77


    tambien borrare la columna user_url ya que voy a usar user_id para referirme al usuario

In [40]:
del df_ui["user_url"]

In [41]:
df_ui.sample(7)

,item_id,item_name,playtime_forever,user_id,items_count
3841412,218620,PAYDAY 2,30,76561197965326033,163
3251625,298600,Lovely Planet,0,CyclonicCyance,83
3613646,244630,NEOTOKYO°,11,SouseisekiChan,390
4883120,230410,Warframe,855,almondkung,69
360320,301970,Screencheat,160,preavus,600
530876,253710,theHunter,74,4mrtm,127
2137084,209870,Blacklight: Retribution,0,badassadmin,114


    con esto reducio bastante el archivo a las columnas que se utilizaran

In [43]:
etf.data_review(df_ui)#llamamos a esta funcion para lograr ver los tipos de datos


Total rows:  5094105

Total full null rows:  0

Total duplicated rows: 13


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,item_id,[<class 'str'>],100.0,5094105,0.0,0
1,item_name,[<class 'str'>],100.0,5094105,0.0,0
2,playtime_forever,[<class 'int'>],100.0,5094105,0.0,0
3,user_id,[<class 'str'>],100.0,5094105,0.0,0
4,items_count,[<class 'int'>],100.0,5094105,0.0,0


tabla para que se vean mejor los tipos 

|columna|tipo|
|-------|----|
|item_id|str|
|item_name|str|
|playtime_forever|int|
|user_id|str|
|items_count|int|

* item_id (numero que identifica al juego nos sirve para relacionar )
* item_name (nombre que identifica al juego nos sirve para relacionar) 
* playtime_forever (tiempo en minutos jugado por un usuario)
* user_id (identificador de usuario puede ser texto o  numerico)


items_count lo dejamos en caso de que lo necesitemos
* items_count (cantidad de items que tiene un usuario nos va a servir para remover filas)#ya no poseemos items count 0 ya que no poseemos valores nulos

    item id lo dejare como texto para luego relacionarlo asi que abajo verificare  si todos sus valores "texto" son numericos con una funcion

In [42]:
etf.valores_no_numericos(df_ui["item_id"])#devuelve una lista con los valores no numericos que se encuentren en una columna

[]

    en este caso devolvio una lista vacia lo que significa que no hay valores que no sean numeros en item_id

## guardar el archivo en parquet

In [47]:
df_ui.to_parquet(r"../Datasets/df_uitems.parquet")

<hr>

# analisis de sentimiento

    volvemos a cargar el archivo dict reviews que creamos mas arriba para agregarle la columna de analisis de sentimiento que nos servira para varias funciones

In [73]:
from textblob import TextBlob

In [72]:
df_ur = pd.read_parquet(r"../Datasets/reviews_dict.parquet")

los valores que tomara esta nueva columna seran los siguientes:

|valor|representa|
|-----|----------|
|0    |negativo  |
|1    |neutral   |
|2    |positivo  |

    si un review no tiene texto en la parte de comentario sera considerado neutral

In [74]:
# funcion lambda que crea una columna "sentimiento" que en caso de que la review este vacia le setea valor 0 si no guarda en esta columna el valor de la polaridad
df_ur['sentimiento'] = df_ur['review'].apply(lambda x: 1 if x == "" else TextBlob(x).sentiment.polarity)

# 0 negativo si la polaridad es < a 0 , 1 si la polaridad es == 0 de otro modo significa que polaridad es mayor a 0 le damos el valor de 2 para positivo
df_ur['sentimiento'] = df_ur['sentimiento'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))

    con esto ya tenemos la columna sentimiento con los valores que establecimos en la tabla superior

In [75]:
df_ur

,posted,item_id,recommend,review,sentimiento
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2
0,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...
2,Posted July 10.,70,True,a must have classic from steam definitely wort...,2
3,Posted July 8.,362890,True,this game is a perfect remake of the original ...,2
0,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,2
1,Posted July 20.,730,True,:D,2


    y dropeare review ya que ya no lo necesitamos

In [76]:
del df_ur["review"]

    ya contamos con un dataframe de pocas columnas y con los valores que necesitamos asi que sobrescribire el primer archivo por este con la columna sentimiento

In [77]:
df_ur

,posted,item_id,recommend,sentimiento
0,"Posted November 5, 2011.",1250,True,2
1,"Posted July 15, 2011.",22200,True,2
2,"Posted April 21, 2011.",43110,True,2
0,"Posted June 24, 2014.",251610,True,2
1,"Posted September 8, 2013.",227300,True,0
...,...,...,...,...
2,Posted July 10.,70,True,2
3,Posted July 8.,362890,True,2
0,Posted July 3.,273110,True,2
1,Posted July 20.,730,True,2


## guardar en archivo parquet

In [ ]:
df_ur.to_parquet(r"../Datasets/reviews_dict.parquet")

<hr>

# archivos especificos

### creando el archivo df_recommend.parquet

este archivo sera el archivo con el que la api trabajara estas funciones:

#### "UsersRecommend"

es una funcion que recibe un año y devuelve el top 3 juegos más recomendados por usuarios para el año dado.(teniendo en cuenta las cantidades de recommendaciones(True) y comentarios positivos y neutrales(0 y 2))

    #Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

#### "UsersWorstDeveloper"

la cual va recibir año y devuelve el top 3 desarrolladoras con juegos menos recomendados para ese año (teniendo en cuenta el total de recommend igual a false y reseñas negativas )

    #Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

cargamos los archivos que utilizaremos para generar el archivo df_recommend.parquet

In [184]:
df_rd = pd.read_parquet(r"../Datasets/reviews_dict.parquet")
df_sg = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json" ,lines=True)

In [185]:
df_rd

,posted,item_id,recommend,sentimiento
0,"Posted November 5, 2011.",1250,True,2
1,"Posted July 15, 2011.",22200,True,2
2,"Posted April 21, 2011.",43110,True,2
0,"Posted June 24, 2014.",251610,True,2
1,"Posted September 8, 2013.",227300,True,0
...,...,...,...,...
2,Posted July 10.,70,True,2
3,Posted July 8.,362890,True,2
0,Posted July 3.,273110,True,2
1,Posted July 20.,730,True,2


cambio  item_id a float para relacionarlo con id ya que id se encuentra en float y no me permite cambiarlo a si que para no complicarse tanto

In [16]:
df_rd["item_id"] = df_rd["item_id"].astype(float)

In [186]:
df_rd

,posted,item_id,recommend,sentimiento
0,"Posted November 5, 2011.",1250,True,2
1,"Posted July 15, 2011.",22200,True,2
2,"Posted April 21, 2011.",43110,True,2
0,"Posted June 24, 2014.",251610,True,2
1,"Posted September 8, 2013.",227300,True,0
...,...,...,...,...
2,Posted July 10.,70,True,2
3,Posted July 8.,362890,True,2
0,Posted July 3.,273110,True,2
1,Posted July 20.,730,True,2


In [187]:
df_sg

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [6]:
del df_sg["genres"]
del df_sg["price"]
del df_sg["release_date"]
del df_sg["reviews_url"]
del df_sg["url"]
del  df_sg["early_access"]
del  df_sg["specs"]
del  df_sg["tags"]
del  df_sg["title"]
del df_sg["publisher"]

In [9]:
df_sg

,app_name,id,developer
0,None,NaN,None
1,None,NaN,None
2,None,NaN,None
3,None,NaN,None
4,None,NaN,None
...,...,...,...
120440,Colony On Mars,773640.0,"Nikita ""Ghost_RUS"""
120441,LOGistICAL: South Africa,733530.0,Sacada
120442,Russian Roads,610660.0,Laush Dmitriy Sergeevich
120443,EXIT 2 - Directions,658870.0,"xropi,stev3ns"


In [10]:
df_sg.drop_duplicates(inplace=True)

In [11]:
df_sg.head(3)

,app_name,id,developer
0,None,NaN,None
88310,Lost Summoner Kitty,761140.0,Kotoshiro
88311,Ironbound,643980.0,Secret Level SRL


In [12]:
df_sg.drop(0,inplace=True)#borramos la unica columna que no poseia datos que quedaba

In [13]:
df_sg.head(3)

,app_name,id,developer
88310,Lost Summoner Kitty,761140.0,Kotoshiro
88311,Ironbound,643980.0,Secret Level SRL
88312,Real Pool 3D - Poolians,670290.0,Poolians.com


In [18]:
merge = df_rd.merge(df_sg,how="inner",left_on="item_id",right_on="id")

In [19]:
merge

,posted,item_id,recommend,sentimiento,app_name,id,developer
0,"Posted November 5, 2011.",1250.0,True,2,Killing Floor,1250.0,Tripwire Interactive
1,"Posted March 30, 2015.",1250.0,True,2,Killing Floor,1250.0,Tripwire Interactive
2,"Posted July 12, 2013.",1250.0,True,2,Killing Floor,1250.0,Tripwire Interactive
3,"Posted August 13, 2015.",1250.0,True,1,Killing Floor,1250.0,Tripwire Interactive
4,"Posted April 5, 2014.",1250.0,True,1,Killing Floor,1250.0,Tripwire Interactive
...,...,...,...,...,...,...,...
53185,"Posted October 29, 2015.",307130.0,True,2,Asteria,307130.0,Legend Studio
53186,Posted May 5.,209120.0,True,0,Street Fighter X Tekken,209120.0,"Capcom U.S.A., Inc."
53187,Posted August 13.,220090.0,True,0,The Journey Down: Chapter One,220090.0,SkyGoblin
53188,Posted August 13.,262850.0,True,0,The Journey Down: Chapter Two,262850.0,SkyGoblin


In [21]:
etf.data_review(merge)


Total rows:  53190

Total full null rows:  0

Total duplicated rows: 8025


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,posted,[<class 'str'>],100.00,53190,0.00,0
1,item_id,[<class 'float'>],100.00,53190,0.00,0
2,recommend,[<class 'bool'>],100.00,53190,0.00,0
3,sentimiento,[<class 'int'>],100.00,53190,0.00,0
4,app_name,[<class 'str'>],100.00,53190,0.00,0
5,id,[<class 'float'>],100.00,53190,0.00,0
6,developer,"[<class 'str'>, <class 'NoneType'>]",93.29,49619,6.71,3571


    en este caso no borramos las columnas duplicadas ya que al no tener la columna review es muy probable que haya filas repetidas ya que son muy pocas columnas(muy pocos valores posibles) y estos serian registros que de tener el comentario(review) serian distintos #los duplicaados ya los borramos antes de quitar la columna  review

lo guardamos como df_recommend.parquet

In [20]:
merge.to_parquet(r"../app/df_recommend.parquet")

<hr>

### creando df_sentimientos.parquet

este archivo se usar paraa la funcion:

## "sentiment_analysis"

recibe el nombre de una empresa y segun la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

    Ejemplo de retorno:{'Valve': {'Negative': 182, 'Neutral': 120, 'Positive': 278}}


In [ ]:
df_sg = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json" ,lines=True) #archivo steam_games

    transformo las siguientes columnas a tipo str ya que es dificil manejar listas y tira errores en bastantes metodos de pandas

In [ ]:
df_sg["genres"] = df_sg["genres"].astype(str)
df_sg["tags"] = df_sg["tags"].astype(str)
df_sg["specs"] = df_sg["specs"].astype(str)

In [ ]:
etf.data_review(df_sg)

    para esta funcion solo utilizaremos(en este df):
* id(para relacionar con la review)
* developer(va a ser por lo que vamos a filtrar/el dato que se pasa a la funcion)

se podria usar pero no la usare:
* publisher(en caso de no tener developer podria devolver esto como publisher y no como developer) #pero la funcion pide developer asi que no la usare

In [ ]:
del df_sg["genres"]
del df_sg["app_name"]
del df_sg["title"]
del df_sg["url"]
del df_sg["release_date"]
del df_sg["tags"]
del df_sg["reviews_url"]
del df_sg["specs"]
del df_sg["price"]
del df_sg["early_access"]
del df_sg["publisher"]

    borramos todas estas columnas que no vamos a utilizar

## este es nuestro dataframe ahora
contiene las columnas que utilizare ,
pero sigue teniendo filas que no nos sirven

In [ ]:
df_sg

In [ ]:
etf.data_review(df_sg)

    con esto podemos ver la cantidad de null y filas duplicadas que en este caso son las mismas cantidades

## eliminacion de duplicados

    cualquier registro duplicado no resulta importante debido a que los datos ya se encuentran en el dataset 
    (en especial en este caso que tenemos muchas columnas que tienen todas la columnas en null)

In [ ]:
df_sg.drop_duplicates(inplace=True)#elimino todos los duplicados dejando solo registros unicos y con ellos una columna llena de valores null que removere en la siguiente instruccion

    al dropear duplicados nos dejo con cada registro unico incluyendo  una de las lineas con todos sus valores nulos

In [ ]:
df_sg.head(3)

    como en este caso quedo en la primera linea la dropeare indicando el indice

In [ ]:
df_sg.drop(0,inplace=True)#borramos la unica columna que no poseia datos que quedaba

In [ ]:
df_sg.head()

In [ ]:
etf.data_review(df_sg)

    ahora ya no tenemos filas completas con nulos o filas duplicadas lo que no quiere decir que no haya nulos
    como podemos comprobar en la ultima columna de el retorno de etf.data_review donde nos dice
|columna|cantidad de nulos|
|-------|-----------------|
|id| 1
|developer| 3298


## eliminacion de nulos

    borrare cualquier nulo en developer

In [ ]:
mask = df_sg["developer"].isna() #

In [ ]:
df_sg = df_sg[~mask] # uso ~ para invertir la condicion de el  filtro lo que traera todos los que developer no es nulo

    ahora tratare el nulo de id

In [ ]:
df_sg.isna().sum()

In [ ]:
df_sg.dropna(inplace=True)

In [ ]:
etf.data_review(df_sg)

In [ ]:
df_sg["id"] = df_sg["id"].astype(int)

In [ ]:
df_sg["id"] = df_sg["id"].astype(str)

    paso id a str para realizar  el merge

In [ ]:
df_sg["id"] = df_sg["id"].astype(str)

In [ ]:
df_rd = pd.read_parquet("../app/reviews_dict.parquet") #este archivo contiene las reviews y analisis de sentimiento

In [ ]:
df_rd

    en este caso solo usarenos dos columnas:
* item_id(para relacionarlo con id)
* sentimiento(para sacar las cantidades)

In [ ]:
del df_rd["posted"]
del df_rd["recommend"]
del df_rd["review"]

In [ ]:
df_rd["item_id"] = df_rd["item_id"].astype(str)

In [ ]:
df_rd

    con esto conservamos las columnas que necesitamos

In [ ]:
etf.data_review(df_rd)

    podemos notar  que esta perfectamente limpio esto debido a que el archivo review_dict es un archivo ya procesado que obtuvimos de user_reviews

In [ ]:
merge = df_rd.merge(df_sg,how="inner",left_on="item_id",right_on="id")

In [ ]:
merge

## guardamos el archivo

In [ ]:
merge.to_parquet(r"../app/df_sentimientos.parquet")

<hr>

### creando df_genero.parquet

este archivo sera con  el que la api trabajara estas funciones:

#### "PlayTimeGenre

esta funcion recibe un genero(texto) y debe devolver año con horas mas jugadas para ese genero

    ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

#### "UserForGenre"

esta funcion recibe  un genero y devuelve el usuario que acumula mas horas jugadas para ese genero y una lista de la acumulacion de horas jugadas por año

    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}


importamos los archivos que utilizaremos para crear el archivo para estas funciones

In [32]:
df_sg = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json",lines=True)

In [33]:
df_ui = pd.read_parquet(r"../Datasets/df_uitems.parquet")

In [34]:
df_sg

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


para estas funciones nenesitamos:
* genero(genres) valor que voy a recibir para aplicar los filtros
* tags la utilizare para rellenar algunos valores en genres
* año(release_date) voya tomar como año en que jugo el año en que salio el juego ya que esta informacion no se encuentra

    para relacionarla podria utilizar tanto item id como item_name pero como no se me pide el nombre de eljuego solo el genero solo conservare
* item_id


    borro las columnas que no voy a utilizar

In [35]:
del df_sg["publisher"]
del df_sg["app_name"]
del df_sg["title"]
del df_sg["url"]
del df_sg["reviews_url"]
del df_sg["specs"] #esta ya explique por que no la usaria en el eda
del df_sg["price"]
del df_sg["early_access"]
del df_sg["developer"]

In [36]:
df_sg

,genres,release_date,tags,id
0,None,None,None,NaN
1,None,None,None,NaN
2,None,None,None,NaN
3,None,None,None,NaN
4,None,None,None,NaN
...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",2018-01-04,"[Strategy, Indie, Casual, Simulation]",773640.0
120441,"[Casual, Indie, Strategy]",2018-01-04,"[Strategy, Indie, Casual]",733530.0
120442,"[Indie, Racing, Simulation]",2018-01-04,"[Indie, Simulation, Racing]",610660.0
120443,"[Casual, Indie]",2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",658870.0


    convierto genres y tags a str para poder tratar los duplicados y los nulos

In [37]:
df_sg["genres"] = df_sg["genres"].astype(str)
df_sg["tags"] = df_sg["tags"].astype(str)

In [38]:
etf.data_review(df_sg)


Total rows:  120445

Total full null rows:  0

Total duplicated rows: 88311


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genres,[<class 'str'>],100.00,120445,0.00,0
1,release_date,"[<class 'NoneType'>, <class 'str'>]",24.96,30068,75.04,90377
2,tags,[<class 'str'>],100.00,120445,0.00,0
3,id,[<class 'float'>],26.68,32133,73.32,88312


    como vemos una gran cantidad de colummas duplicadas voy a empezar por ahi

## eliminacion de duplicados

In [39]:
df_sg.drop_duplicates(inplace=True)

    al eliminar duplicados nos deja con una sola copia de la columna que estaba llena de nulos

In [40]:
df_sg.head(3)

,genres,release_date,tags,id
0,None,None,None,NaN
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140.0
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",643980.0


In [41]:
df_sg.drop(0,inplace=True)

In [42]:
df_sg.head(3)

,genres,release_date,tags,id
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140.0
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",643980.0
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",670290.0


In [43]:
etf.data_review(df_sg)


Total rows:  32133

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genres,[<class 'str'>],100.00,32133,0.00,0
1,release_date,"[<class 'str'>, <class 'NoneType'>]",93.57,30067,6.43,2066
2,tags,[<class 'str'>],100.00,32133,0.00,0
3,id,[<class 'float'>],100.00,32132,0.00,1


    ahora que convertimos a listas nos dice que  genres  y  tags no tienen nulos por que sus listas vacias se volvieron un texto como este "[]"
    por lo que tendremos que tratar los nulos como texto

In [44]:
df_sg["genres"] = df_sg["genres"].str.strip("[]").str.replace(", '",",").str.replace("'","")
df_sg["tags"] = df_sg["tags"].str.strip("[]").str.replace(", '",",").str.replace("'","")

In [45]:
generos = etf.valores_lista(df_sg,"genres")
tags = etf.valores_lista(df_sg,"tags")

In [46]:
generos

['Simulation',
 'Strategy',
 'Software Training',
 'Design &amp; Illustration',
 'Web Publishing',
 'Casual',
 'Free to Play',
 'Accounting',
 'Utilities',
 'Sports',
 'Education',
 'Animation &amp; Modeling',
 'Photo Editing',
 'Indie',
 'Early Access',
 'None',
 'RPG',
 'Adventure',
 'Audio Production',
 'Action',
 'Video Production',
 'Racing',
 'Massively Multiplayer']

In [47]:
tags

['Bikes',
 ' "Beat em up"',
 'Typing',
 'Music-Based Procedural Generation',
 'Shooter',
 'Trading',
 'Cyberpunk',
 'Retro',
 'Relaxing',
 'Turn-Based Combat',
 'Time Attack',
 'Steam Machine',
 'Early Access',
 'Linear',
 'Sokoban',
 'Otome',
 'MOBA',
 'On-Rails Shooter',
 'Transhumanism',
 'Exploration',
 'Survival Horror',
 'Fighting',
 'Management',
 'Character Action Game',
 'Sailing',
 'Stylized',
 'Choices Matter',
 'Online Co-Op',
 '"Beat em up"',
 'Space Sim',
 'Party-Based RPG',
 'God Game',
 'Agriculture',
 'Post-apocalyptic',
 'Aliens',
 'Atmospheric',
 'Mystery',
 'CRPG',
 'Touch-Friendly',
 'Steampunk',
 'Replay Value',
 'Satire',
 'NSFW',
 'Perma Death',
 'Capitalism',
 'Metroidvania',
 'Puzzle-Platformer',
 'Cycling',
 'Star Wars',
 'Great Soundtrack',
 'Loot',
 'Football',
 'Team-Based',
 'Horses',
 'Mature',
 'Bowling',
 'Gore',
 'Resource Management',
 'Hex Grid',
 'Conversation',
 'LEGO',
 'MMORPG',
 'Female Protagonist',
 'Lovecraftian',
 'Real-Time with Pause',
 '

In [48]:
df_sg

,genres,release_date,tags,id
88310,"Action,Casual,Indie,Simulation,Strategy",2018-01-04,"Strategy,Action,Indie,Casual,Simulation",761140.0
88311,"Free to Play,Indie,RPG,Strategy",2018-01-04,"Free to Play,Strategy,Indie,RPG,Card Game,Trad...",643980.0
88312,"Casual,Free to Play,Indie,Simulation,Sports",2017-07-24,"Free to Play,Simulation,Sports,Casual,Indie,Mu...",670290.0
88313,"Action,Adventure,Casual",2017-12-07,"Action,Adventure,Casual",767400.0
88314,None,None,"Action,Indie,Casual,Sports",773570.0
...,...,...,...,...
120440,"Casual,Indie,Simulation,Strategy",2018-01-04,"Strategy,Indie,Casual,Simulation",773640.0
120441,"Casual,Indie,Strategy",2018-01-04,"Strategy,Indie,Casual",733530.0
120442,"Indie,Racing,Simulation",2018-01-04,"Indie,Simulation,Racing",610660.0
120443,"Casual,Indie",2017-09-02,"Indie,Casual,Puzzle,Singleplayer,Atmospheric,R...",658870.0


In [49]:
generos = etf.valores_lista(df_sg,"genres") #obtengo todos los valores unicos de genres
tags = etf.valores_lista(df_sg,"tags")      #obtengo todos  los valores unicos de tags

elementos_comunes = set(generos) & set(tags) #obtengo la interseccion entre esos valores


In [50]:
elementos_comunes

{'Action',
 'Adventure',
 'Audio Production',
 'Casual',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'None',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing'}

    estos son los valores que comparten generos como tags podemos notar que se encuentra el valor "None"(lo borrare) y algunos valores que podrian no considerarse generos pero  los dejaremos asi yaa que podria  ser que dentro de la plataforma si se los clasifique como generos

In [51]:
elementos_comunes.remove("None")# removemos "None"de la lista  de elementos en comun

In [52]:
elementos_comunes

{'Action',
 'Adventure',
 'Audio Production',
 'Casual',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing'}

    genres antes de rellenar

In [53]:
df_sg["genres"]

88310         Action,Casual,Indie,Simulation,Strategy
88311                 Free to Play,Indie,RPG,Strategy
88312     Casual,Free to Play,Indie,Simulation,Sports
88313                         Action,Adventure,Casual
88314                                            None
                             ...                     
120440               Casual,Indie,Simulation,Strategy
120441                          Casual,Indie,Strategy
120442                        Indie,Racing,Simulation
120443                                   Casual,Indie
120444                                           None
Name: genres, Length: 32133, dtype: object

    llamamos esta funcion que hice en ETL_funciones.py que lo que hace basicamente recibe el dataframe y columna de un dataframe en el cual se van a rellenar los valores "None" con  un segundo data frame y columna que  va a recibir donde va a comparar si un valor de la segunda columna esta en la lista de valores comunes entre ambas columnas rellena este valor en la primera columna 

In [54]:
etf.rellenar_comunes(df_sg,"genres",df_sg,"tags",elementos_comunes)

'fin de ejecucion'

    genres despues de rellenar

In [55]:
df_sg["genres"]

88310            Action,Casual,Indie,Simulation,Strategy
88311                    Free to Play,Indie,RPG,Strategy
88312        Casual,Free to Play,Indie,Simulation,Sports
88313                            Action,Adventure,Casual
88314                         Indie,Sports,Casual,Action
                               ...                      
120440                  Casual,Indie,Simulation,Strategy
120441                             Casual,Indie,Strategy
120442                           Indie,Racing,Simulation
120443                                      Casual,Indie
120444    Simulation,Action,Indie,Early Access,Adventure
Name: genres, Length: 32133, dtype: object

In [56]:
df_sg

,genres,release_date,tags,id
88310,"Action,Casual,Indie,Simulation,Strategy",2018-01-04,"Strategy,Action,Indie,Casual,Simulation",761140.0
88311,"Free to Play,Indie,RPG,Strategy",2018-01-04,"Free to Play,Strategy,Indie,RPG,Card Game,Trad...",643980.0
88312,"Casual,Free to Play,Indie,Simulation,Sports",2017-07-24,"Free to Play,Simulation,Sports,Casual,Indie,Mu...",670290.0
88313,"Action,Adventure,Casual",2017-12-07,"Action,Adventure,Casual",767400.0
88314,"Indie,Sports,Casual,Action",None,"Action,Indie,Casual,Sports",773570.0
...,...,...,...,...
120440,"Casual,Indie,Simulation,Strategy",2018-01-04,"Strategy,Indie,Casual,Simulation",773640.0
120441,"Casual,Indie,Strategy",2018-01-04,"Strategy,Indie,Casual",733530.0
120442,"Indie,Racing,Simulation",2018-01-04,"Indie,Simulation,Racing",610660.0
120443,"Casual,Indie",2017-09-02,"Indie,Casual,Puzzle,Singleplayer,Atmospheric,R...",658870.0


    borramos la columna tags ahora que ya la utilizamos para rellenar  los valores de genres

In [57]:
del df_sg["tags"]

    este es el estado actual de nuestro dataframe

In [58]:
etf.data_review(df_sg)


Total rows:  32133

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genres,[<class 'str'>],100.00,32133,0.00,0
1,release_date,"[<class 'str'>, <class 'NoneType'>]",93.57,30067,6.43,2066
2,id,[<class 'float'>],100.00,32132,0.00,1


    dropeo todas las filas de release date con valores nulos ya que no eh pensado  como rellenarlas y son 2000 valores y de  dropearlos aun asi me quedan 30000

In [59]:
df_sg.dropna(subset=['release_date'],inplace=True)

In [60]:
df_sg.dropna(subset=['id'],inplace=True)

In [61]:
etf.data_review(df_sg)


Total rows:  30066

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genres,[<class 'str'>],100.0,30066,0.0,0
1,release_date,[<class 'str'>],100.0,30066,0.0,0
2,id,[<class 'float'>],100.0,30066,0.0,0


## cambiando release_date por los años que es lo que utilizaremos

In [62]:
import re

In [63]:
pattern = r'\b\d{4}\b'#expresion regular donde  haya 4 numeros juntos 

# Función para aplicar la expresión regular
def obtener_año(value):
    match = re.search(pattern, str(value))
    if match:
        return match.group()
    else:
        return value

# Aplicar la función a la columna release_date
df_sg['release_date'] = df_sg['release_date'].apply(obtener_año)

# Mostrar el resultado
df_sg

,genres,release_date,id
88310,"Action,Casual,Indie,Simulation,Strategy",2018,761140.0
88311,"Free to Play,Indie,RPG,Strategy",2018,643980.0
88312,"Casual,Free to Play,Indie,Simulation,Sports",2017,670290.0
88313,"Action,Adventure,Casual",2017,767400.0
88315,"Action,Adventure,Simulation",2018,772540.0
...,...,...,...
120439,"Action,Adventure,Casual,Indie",2018,745400.0
120440,"Casual,Indie,Simulation,Strategy",2018,773640.0
120441,"Casual,Indie,Strategy",2018,733530.0
120442,"Indie,Racing,Simulation",2018,610660.0


    uso la funcion que cree para verificar si un valor se puede pasar a numerico

In [64]:
no_anio = etf.valores_no_numericos(df_sg["release_date"])#la funcion devuelveuna lista y la guardo en una variable

In [65]:
no_anio#estos son todos los valores que habia en año release date en los que no pudimos obtener el año

['Soon..',
 'Beta测试已开启',
 'Coming Soon',
 'TBA',
 "When it's done",
 'coming soon',
 'soon',
 '14 July',
 'To be Announced',
 'TBD',
 'Soon',
 '2018年初頭発売予定',
 'Please wait warmly',
 'early access',
 'SOON',
 'Coming Soon/Próximamente',
 '预热群52756441',
 '2018年1月',
 'Coming soon',
 '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
 '"""Soon"""',
 'Demo coming soon.',
 'Not yet available',
 "C'est bientôt...                    (ou pas)",
 'Datachunks conflicted. Be vigiliant.',
 'Demo is available now!',
 'Play Beta in demo!',
 'Coming Soon!',
 'To be announced',
 'When it is finished',
 'Release Date TBA',
 'When it is ready',
 'Early Access Starting Soon!',
 'Comming Soon',
 'Early Access soon',
 'To Be Announced',
 'Coming soon..',
 'Soon™',
 "When it's done!",
 'SOON™']

    borramos las filas con estos valores en release date

In [66]:
df_sg = df_sg[~df_sg['release_date'].isin(no_anio)]# filtrando el df en donde release_date NO tiene los valores que obtuvimos que no se podian pasar a numerico y sobrescribimos el dataframe con este sin esos valores

    si volvemos a llamar a la funcion nos devuelve que ya no posee valores no numericos y como aplicamos la expresion regular solo  tendremos valores de 4 numeros lo que no evita que haya un 0000 un 9999 pero es muy poco posible

In [68]:
etf.valores_no_numericos(df_sg["release_date"])

[]

    despues de este proceso perdimos alrededor de 100  filas pero eran filas que no nos servian

In [70]:
etf.data_review(df_sg)


Total rows:  29962

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genres,[<class 'str'>],100.0,29962,0.0,0
1,release_date,[<class 'str'>],100.0,29962,0.0,0
2,id,[<class 'float'>],100.0,29962,0.0,0


### cambiamos el tipo de dato de release_date a tipo fecha año

In [71]:
df_sg['release_date'] = pd.to_datetime(df_sg['release_date'], errors='coerce').dt.year

C:\Users\delba\AppData\Local\Temp\ipykernel_9840\3544135433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sg['release_date'] = pd.to_datetime(df_sg['release_date'], errors='coerce').dt.year


    estas seran todas las trasformaciones paraa este primer archivo

<hr>

In [72]:
df_ui

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,277
2,30,Day of Defeat,7,76561197970982479,277
3,40,Deathmatch Classic,0,76561197970982479,277
4,50,Half-Life: Opposing Force,0,76561197970982479,277
...,...,...,...,...,...
5153204,346330,BrainBread 2,0,76561198329548331,7
5153205,373330,All Is Dust,0,76561198329548331,7
5153206,388490,One Way To Die: Steam Edition,3,76561198329548331,7
5153207,521570,You Have 10 Seconds 2,4,76561198329548331,7


    en este dataframe no utilizare items_count ni item_name

In [73]:
del df_ui["item_name"]
del df_ui["items_count"]

In [74]:
df_ui

,item_id,playtime_forever,user_id
0,10,6,76561197970982479
1,20,0,76561197970982479
2,30,7,76561197970982479
3,40,0,76561197970982479
4,50,0,76561197970982479
...,...,...,...
5153204,346330,0,76561198329548331
5153205,373330,0,76561198329548331
5153206,388490,3,76561198329548331
5153207,521570,4,76561198329548331


    ademas removere cualquier registro  con playtime_forever con valor 0

In [75]:
mask =  (df_ui["playtime_forever"] ==  0)#filtro que busca donde los valores de tiempo de juego son 0

In [76]:
df_ui = df_ui[~mask]  #utilizo el caracter ~ para que la mascara me traiga los tiempo de juego que NO son 0

In [77]:
df_ui

,item_id,playtime_forever,user_id
0,10,6,76561197970982479
2,30,7,76561197970982479
8,300,4733,76561197970982479
9,240,1853,76561197970982479
10,3830,333,76561197970982479
...,...,...,...
5153202,304930,677,76561198329548331
5153203,227940,43,76561198329548331
5153206,388490,3,76561198329548331
5153207,521570,4,76561198329548331


    este  es el resultado de el dataframe sin tiempo de juego 0

In [78]:
etf.data_review(df_ui)


Total rows:  3246375

Total full null rows:  0

Total duplicated rows: 13


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,item_id,[<class 'str'>],100.0,3246375,0.0,0
1,playtime_forever,[<class 'int'>],100.0,3246375,0.0,0
2,user_id,[<class 'str'>],100.0,3246375,0.0,0


    tenemos 13 filas duplicadas lo que no puede ser yaa que si ubiese alguna duplicada serian 2 registros diferentes de un usuario para un mismo juego

In [79]:
df_ui.drop_duplicates(inplace=True)#borro los duplicados

C:\Users\delba\AppData\Local\Temp\ipykernel_9840\1268282121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ui.drop_duplicates(inplace=True)#borro los duplicados


In [80]:
etf.data_review(df_ui)


Total rows:  3246362

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,item_id,[<class 'str'>],100.0,3246362,0.0,0
1,playtime_forever,[<class 'int'>],100.0,3246362,0.0,0
2,user_id,[<class 'str'>],100.0,3246362,0.0,0


    cambio item_id a float para hacer el merge

In [81]:
df_ui["item_id"] = df_ui["item_id"].astype(float)

C:\Users\delba\AppData\Local\Temp\ipykernel_9840\1376868486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ui["item_id"] = df_ui["item_id"].astype(float)


    este  dataframe ya se encuentra listo para el merge

<hr>

In [82]:
df_genero = df_sg.merge(df_ui,how="inner",left_on="id",right_on="item_id")   

In [83]:
df_genero.to_parquet(r"../app/df_genero.parquet")

<hr>

# archivo para el modelo

para crear el modelo me vot a limitar a tener item_id , genres  y app_name

    lo hare a partir de el archivo de steam_games

In [157]:
df_sg =  pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json", lines=True)

In [158]:
df_sg

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


borramos todas  las columnas  queno usaremos y dropeamos duplicados

In [159]:
del df_sg["publisher"]
del df_sg["title"]#tiene muchos valores nulos a diferencia de app_name que tiene solo 2
del df_sg["url"]
del df_sg["release_date"]
del df_sg["reviews_url"]
del df_sg["specs"]
del df_sg["price"]
del df_sg["early_access"]
del df_sg["developer"]

In [161]:
df_sg

,genres,app_name,tags,id
0,None,None,None,NaN
1,None,None,None,NaN
2,None,None,None,NaN
3,None,None,None,NaN
4,None,None,None,NaN
...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,"[Strategy, Indie, Casual, Simulation]",773640.0
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,"[Strategy, Indie, Casual]",733530.0
120442,"[Indie, Racing, Simulation]",Russian Roads,"[Indie, Simulation, Racing]",610660.0
120443,"[Casual, Indie]",EXIT 2 - Directions,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",658870.0


    convierto genres y tags a str para poder tratar los duplicados y los nulos

In [162]:
df_sg["genres"] = df_sg["genres"].astype(str)
df_sg["tags"] = df_sg["tags"].astype(str)

In [163]:
etf.data_review(df_sg)


Total rows:  120445

Total full null rows:  0

Total duplicated rows: 88311


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genres,[<class 'str'>],100.00,120445,0.00,0
1,app_name,"[<class 'NoneType'>, <class 'str'>]",26.68,32133,73.32,88312
2,tags,[<class 'str'>],100.00,120445,0.00,0
3,id,[<class 'float'>],26.68,32133,73.32,88312


    como emos una gran cantidad de colummas duplicadas voy a empezar por ahi

## eliminacion de duplicados

In [164]:
df_sg.drop_duplicates(inplace=True)

    al eliminar duplicados nos deja con una sola copia de la columna que estaba llena de nulos

In [165]:
df_sg.head(3)

,genres,app_name,tags,id
0,None,None,None,NaN
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140.0
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",643980.0


In [166]:
df_sg.drop(0,inplace=True)

In [167]:
df_sg.head(3)

,genres,app_name,tags,id
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140.0
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",643980.0
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...",670290.0


In [168]:
etf.data_review(df_sg)


Total rows:  32133

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genres,[<class 'str'>],100.0,32133,0.0,0
1,app_name,"[<class 'str'>, <class 'NoneType'>]",100.0,32132,0.0,1
2,tags,[<class 'str'>],100.0,32133,0.0,0
3,id,[<class 'float'>],100.0,32132,0.0,1


    ahora que convertimos a listas nos dice que  genres  y  tags no tienen nulos por que sus listas vacias se volvieron un texto como este "[]"
    por lo que tendremos que tratar los nulos como texto

In [169]:
df_sg["genres"] = df_sg["genres"].str.strip("[]").str.replace(", '",",").str.replace("'","")
df_sg["tags"] = df_sg["tags"].str.strip("[]").str.replace(", '",",").str.replace("'","")

In [141]:
df_sg.drop_duplicates(inplace=True)

In [170]:
df_sg

,genres,app_name,tags,id
88310,"Action,Casual,Indie,Simulation,Strategy",Lost Summoner Kitty,"Strategy,Action,Indie,Casual,Simulation",761140.0
88311,"Free to Play,Indie,RPG,Strategy",Ironbound,"Free to Play,Strategy,Indie,RPG,Card Game,Trad...",643980.0
88312,"Casual,Free to Play,Indie,Simulation,Sports",Real Pool 3D - Poolians,"Free to Play,Simulation,Sports,Casual,Indie,Mu...",670290.0
88313,"Action,Adventure,Casual",弹炸人2222,"Action,Adventure,Casual",767400.0
88314,None,Log Challenge,"Action,Indie,Casual,Sports",773570.0
...,...,...,...,...
120440,"Casual,Indie,Simulation,Strategy",Colony On Mars,"Strategy,Indie,Casual,Simulation",773640.0
120441,"Casual,Indie,Strategy",LOGistICAL: South Africa,"Strategy,Indie,Casual",733530.0
120442,"Indie,Racing,Simulation",Russian Roads,"Indie,Simulation,Racing",610660.0
120443,"Casual,Indie",EXIT 2 - Directions,"Indie,Casual,Puzzle,Singleplayer,Atmospheric,R...",658870.0


    uso esta funcion en ETL_functions.py para obtener una lista con los valores unicos de cada caategoria y sacar los valores en comun

In [ ]:
generos = etf.valores_lista(df_sg,"genres") #obtengo todos los valores unicos de genres
tags = etf.valores_lista(df_sg,"tags")      #obtengo todos  los valores unicos de tags

elementos_comunes = set(generos) & set(tags) #obtengo la interseccion entre esos valores

In [173]:
elementos_comunes

{'Action',
 'Adventure',
 'Audio Production',
 'Casual',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing'}

    genres antes de rellenar

In [174]:
df_sg["genres"]

88310         Action,Casual,Indie,Simulation,Strategy
88311                 Free to Play,Indie,RPG,Strategy
88312     Casual,Free to Play,Indie,Simulation,Sports
88313                         Action,Adventure,Casual
88314                                            None
                             ...                     
120440               Casual,Indie,Simulation,Strategy
120441                          Casual,Indie,Strategy
120442                        Indie,Racing,Simulation
120443                                   Casual,Indie
120444                                           None
Name: genres, Length: 32133, dtype: object

    llamamos esta funcion que hice en ETL_funciones.py que lo que hace basicamente recibe el dataframe y columna de un dataframe en el cual se van a rellenar los valores "None" con  un segundo data frame y columna que  va a recibir donde va a comparar si un valor de la segunda columna esta en la lista de valores comunes entre ambas columnas rellena este valor en la primera columna

In [175]:
etf.rellenar_comunes(df_sg,"genres",df_sg,"tags",elementos_comunes)

'fin de ejecucion'

    genres despues de rellenar

In [176]:
df_sg["genres"]

88310            Action,Casual,Indie,Simulation,Strategy
88311                    Free to Play,Indie,RPG,Strategy
88312        Casual,Free to Play,Indie,Simulation,Sports
88313                            Action,Adventure,Casual
88314                         Indie,Sports,Casual,Action
                               ...                      
120440                  Casual,Indie,Simulation,Strategy
120441                             Casual,Indie,Strategy
120442                           Indie,Racing,Simulation
120443                                      Casual,Indie
120444    Simulation,Action,Indie,Early Access,Adventure
Name: genres, Length: 32133, dtype: object

In [177]:
df_sg

,genres,app_name,tags,id
88310,"Action,Casual,Indie,Simulation,Strategy",Lost Summoner Kitty,"Strategy,Action,Indie,Casual,Simulation",761140.0
88311,"Free to Play,Indie,RPG,Strategy",Ironbound,"Free to Play,Strategy,Indie,RPG,Card Game,Trad...",643980.0
88312,"Casual,Free to Play,Indie,Simulation,Sports",Real Pool 3D - Poolians,"Free to Play,Simulation,Sports,Casual,Indie,Mu...",670290.0
88313,"Action,Adventure,Casual",弹炸人2222,"Action,Adventure,Casual",767400.0
88314,"Indie,Sports,Casual,Action",Log Challenge,"Action,Indie,Casual,Sports",773570.0
...,...,...,...,...
120440,"Casual,Indie,Simulation,Strategy",Colony On Mars,"Strategy,Indie,Casual,Simulation",773640.0
120441,"Casual,Indie,Strategy",LOGistICAL: South Africa,"Strategy,Indie,Casual",733530.0
120442,"Indie,Racing,Simulation",Russian Roads,"Indie,Simulation,Racing",610660.0
120443,"Casual,Indie",EXIT 2 - Directions,"Indie,Casual,Puzzle,Singleplayer,Atmospheric,R...",658870.0


    borramos la columna tags ahora que ya la utilizamos para rellenar  los valores de genres

In [178]:
del df_sg["tags"]

In [179]:
df_sg

,genres,app_name,id
88310,"Action,Casual,Indie,Simulation,Strategy",Lost Summoner Kitty,761140.0
88311,"Free to Play,Indie,RPG,Strategy",Ironbound,643980.0
88312,"Casual,Free to Play,Indie,Simulation,Sports",Real Pool 3D - Poolians,670290.0
88313,"Action,Adventure,Casual",弹炸人2222,767400.0
88314,"Indie,Sports,Casual,Action",Log Challenge,773570.0
...,...,...,...
120440,"Casual,Indie,Simulation,Strategy",Colony On Mars,773640.0
120441,"Casual,Indie,Strategy",LOGistICAL: South Africa,733530.0
120442,"Indie,Racing,Simulation",Russian Roads,610660.0
120443,"Casual,Indie",EXIT 2 - Directions,658870.0


    como ultimo cambio antes de guardar el dataframe en un archivo voy a cambiar las , en genres  por espacios asi quedan las palbras separadasa yq que voy a usar tfidfvectorizer

In [181]:
df_sg["genres"] =  df_sg["genres"].str.replace(","," ")

ahora si este es nuestro dataframe final

In [182]:
df_sg

,genres,app_name,id
88310,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,761140.0
88311,Free to Play Indie RPG Strategy,Ironbound,643980.0
88312,Casual Free to Play Indie Simulation Sports,Real Pool 3D - Poolians,670290.0
88313,Action Adventure Casual,弹炸人2222,767400.0
88314,Indie Sports Casual Action,Log Challenge,773570.0
...,...,...,...
120440,Casual Indie Simulation Strategy,Colony On Mars,773640.0
120441,Casual Indie Strategy,LOGistICAL: South Africa,733530.0
120442,Indie Racing Simulation,Russian Roads,610660.0
120443,Casual Indie,EXIT 2 - Directions,658870.0


## guardar como archivo.parquet

In [183]:
df_sg.to_parquet(r"../app/df_modelo.parquet")

<hr>

## archivo para el modelo 2

In [14]:
df = pd.read_parquet(r"../app/df_modelo.parquet")

df2 = pd.read_parquet(r"../Datasets/df_uitems.parquet")

    para esto  usare las 2 siguientes dataframe para crear un archivo que me sirva

In [15]:
df

,genres,app_name,id
88310,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,761140.0
88311,Free to Play Indie RPG Strategy,Ironbound,643980.0
88312,Casual Free to Play Indie Simulation Sports,Real Pool 3D - Poolians,670290.0
88313,Action Adventure Casual,弹炸人2222,767400.0
88314,Indie Sports Casual Action,Log Challenge,773570.0
...,...,...,...
120440,Casual Indie Simulation Strategy,Colony On Mars,773640.0
120441,Casual Indie Strategy,LOGistICAL: South Africa,733530.0
120442,Indie Racing Simulation,Russian Roads,610660.0
120443,Casual Indie,EXIT 2 - Directions,658870.0


In [16]:
df2

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,277
2,30,Day of Defeat,7,76561197970982479,277
3,40,Deathmatch Classic,0,76561197970982479,277
4,50,Half-Life: Opposing Force,0,76561197970982479,277
...,...,...,...,...,...
5153204,346330,BrainBread 2,0,76561198329548331,7
5153205,373330,All Is Dust,0,76561198329548331,7
5153206,388490,One Way To Die: Steam Edition,3,76561198329548331,7
5153207,521570,You Have 10 Seconds 2,4,76561198329548331,7


    transformo item_id de df2 a float para relacionar con df ya que es el camino mas rapido para hacer el merge en lugar de normalizar el primero

In [17]:
df2["item_id"] = df2["item_id"].astype(float)

In [18]:
df2

,item_id,item_name,playtime_forever,user_id,items_count
0,10.0,Counter-Strike,6,76561197970982479,277
1,20.0,Team Fortress Classic,0,76561197970982479,277
2,30.0,Day of Defeat,7,76561197970982479,277
3,40.0,Deathmatch Classic,0,76561197970982479,277
4,50.0,Half-Life: Opposing Force,0,76561197970982479,277
...,...,...,...,...,...
5153204,346330.0,BrainBread 2,0,76561198329548331,7
5153205,373330.0,All Is Dust,0,76561198329548331,7
5153206,388490.0,One Way To Die: Steam Edition,3,76561198329548331,7
5153207,521570.0,You Have 10 Seconds 2,4,76561198329548331,7


In [19]:
merge = df.merge(df2,how="inner",left_on="id",right_on="item_id")

In [20]:
merge

,genres,app_name,id,item_id,item_name,playtime_forever,user_id,items_count
0,Action Indie Racing,Carmageddon Max Pack,282010.0,282010.0,Carmageddon Max Pack,5,UTNerd24,188
1,Action Indie Racing,Carmageddon Max Pack,282010.0,282010.0,Carmageddon Max Pack,0,I_DID_911_JUST_SAYING,154
2,Action Indie Racing,Carmageddon Max Pack,282010.0,282010.0,Carmageddon Max Pack,0,76561197962104795,79
3,Action Indie Racing,Carmageddon Max Pack,282010.0,282010.0,Carmageddon Max Pack,0,r3ap3r78,331
4,Action Indie Racing,Carmageddon Max Pack,282010.0,282010.0,Carmageddon Max Pack,13,saint556,583
...,...,...,...,...,...,...,...,...
4244826,Action,Counter-Strike: Condition Zero,80.0,80.0,Counter-Strike: Condition Zero,0,76561198273508956,15
4244827,Action,Counter-Strike: Condition Zero,80.0,80.0,Counter-Strike: Condition Zero,0,76561198282090798,5
4244828,Action,Counter-Strike: Condition Zero,80.0,80.0,Counter-Strike: Condition Zero,0,943525,58
4244829,Action,Counter-Strike: Condition Zero,80.0,80.0,Counter-Strike: Condition Zero,9,76561198283312749,49


voy a dropear items count , id y playtime forever

In [21]:
del merge["id"]
del merge["items_count"]
del merge["playtime_forever"]

In [22]:
merge

,genres,app_name,item_id,item_name,user_id
0,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,UTNerd24
1,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,I_DID_911_JUST_SAYING
2,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,76561197962104795
3,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,r3ap3r78
4,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,saint556
...,...,...,...,...,...
4244826,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,76561198273508956
4244827,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,76561198282090798
4244828,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,943525
4244829,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,76561198283312749


    tambien por el caso practico de poder hacer las recomendaciones sin mucha conplicacion voy a remover cualquier user id que no sea solo valores numericos
    esto se  podria evitar rellenando con valores que coincidan en user_data.parquet pero como tampoco vamos a usar toda la muestra tomare esta opcion

In [23]:
no_num_id = etf.valores_no_numericos(merge["user_id"])#esto tomo varios min ya que son 4244831 registros

In [27]:
merge[merge["user_id"].isin(no_num_id)]# vemos que mas de la mitad de registros cuentan con esta caracteristica pero con el ojetetivo  de hacer el modelo los borraremos

,genres,app_name,item_id,item_name,user_id
0,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,UTNerd24
1,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,I_DID_911_JUST_SAYING
3,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,r3ap3r78
4,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,saint556
5,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,chidvd
...,...,...,...,...,...
4244816,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,supergeroi228
4244818,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,celebrexISGO
4244819,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,Keenno90
4244821,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,steamjke


In [29]:
merge = merge[~merge["user_id"].isin(no_num_id)]#guardamos los valores que no estan en la lista de no_num_id

In [31]:
merge#nos quedan casi 2 millones de registros

,genres,app_name,item_id,item_name,user_id
2,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,76561197962104795
8,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,76561198014848587
11,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,76561198018144075
13,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,76561198001688490
14,Action Indie Racing,Carmageddon Max Pack,282010.0,Carmageddon Max Pack,76561197974288321
...,...,...,...,...,...
4244826,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,76561198273508956
4244827,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,76561198282090798
4244828,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,943525
4244829,Action,Counter-Strike: Condition Zero,80.0,Counter-Strike: Condition Zero,76561198283312749


In [34]:
merge = merge.sample(n=10000, random_state=42)# tomo una muestra aleatoria de 20mil registros

In [35]:
merge

,genres,app_name,item_id,item_name,user_id
3801029,Action,STAR WARS™ Jedi Knight - Jedi Academy™,6020.0,STAR WARS™ Jedi Knight: Jedi Academy™,76561198090150367
3897418,RPG,Fallout 3,22300.0,Fallout 3,76561198081355466
1736609,Action Free to Play Indie Massively Multiplaye...,Robocraft,301520.0,Robocraft,76561198074033911
617609,Action Adventure Indie RPG,The Binding of Isaac,113200.0,The Binding of Isaac,76561198065827093
1509120,Action Adventure Indie,Downwell,360740.0,Downwell,76561198065890292
...,...,...,...,...,...
2261573,Action Strategy,Bloons TD 5,306020.0,Bloons TD5,76561198094513310
2000635,Action Free to Play Early Access,WARMODE,391460.0,WARMODE,76561198075724165
38984,Indie Simulation,Garry's Mod,4000.0,Garry's Mod,76561198097764470
2551978,Action Adventure RPG,South Park™: The Stick of Truth™,213670.0,South Park™: The Stick of Truth™,76561198072917560


In [42]:
merge.to_parquet(r"../app/df_modelo2.parquet")

In [36]:
genres_dummies = merge['genres'].str.get_dummies(sep=' ')#genero un dataframe con los dummies de genero

In [41]:
genres_dummies

,&amp;,Access,Action,Adventure,Animation,Casual,Design,Early,Education,Free,...,Racing,Simulation,Software,Sports,Strategy,Training,Utilities,Video,Web,to
3801029,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3897418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1736609,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
617609,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1509120,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261573,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2000635,0,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
38984,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2551978,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
genres_dummies.to_parquet(r"../app/df_gendumm.parquet")